In [1]:
# Beat tracking example
from __future__ import print_function
import librosa

# 1. Get the file path to the included audio example
filename = librosa.util.example_audio_file()

# 2. Load the audio as a waveform `y`
#    Store the sampling rate as `sr`
y, sr = librosa.load(filename)

# 3. Run the default beat tracker
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)

print('Estimated tempo: {:.2f} beats per minute'.format(tempo))

# 4. Convert the frame indices of beat events into timestamps
beat_times = librosa.frames_to_time(beat_frames, sr=sr)

/usr/local/lib/python3.6/dist-packages/librosa/beat.py:306: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  hop_length=hop_length))


Estimated tempo: 129.20 beats per minute


In [2]:
beat_times

array([ 0.11609977,  0.55727891,  0.99845805,  1.46285714,  1.92725624,
        2.39165533,  2.83283447,  3.29723356,  3.76163265,  4.22603175,
        4.69043084,  5.15482993,  5.61922902,  6.08362812,  6.52480726,
        6.98920635,  7.45360544,  7.91800454,  8.38240363,  8.87002268,
        9.31120181,  9.77560091, 10.24      , 10.70439909, 11.14557823,
       11.60997732, 12.07437642, 12.53877551, 13.0031746 , 13.4675737 ,
       13.93197279, 14.39637188, 14.83755102, 15.27873016, 15.74312925,
       16.20752834, 16.67192744, 17.11310658, 17.60072562, 18.04190476,
       18.52952381, 18.97070295, 19.43510204, 19.89950113, 20.36390023,
       20.80507937, 21.29269841, 21.73387755, 22.2214966 , 22.66267574,
       23.12707483, 23.59147392, 24.05587302, 24.49705215, 24.96145125,
       25.42585034, 25.91346939, 26.35464853, 26.81904762, 27.28344671,
       27.7478458 , 28.18902494, 28.65342404, 29.11782313, 29.60544218,
       30.06984127, 30.53424036, 30.9754195 , 31.43981859, 31.88

In [3]:
# Feature extraction example
import numpy as np
import librosa

# Load the example clip
y, sr = librosa.load(librosa.util.example_audio_file())

# Set the hop length; at 22050 Hz, 512 samples ~= 23ms
hop_length = 512

# Separate harmonics and percussives into two waveforms
y_harmonic, y_percussive = librosa.effects.hpss(y)

# Beat track on the percussive signal
tempo, beat_frames = librosa.beat.beat_track(y=y_percussive,
                                             sr=sr)

# Compute MFCC features from the raw signal
mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)

# And the first-order differences (delta features)
mfcc_delta = librosa.feature.delta(mfcc)

# Stack and synchronize between beat events
# This time, we'll use the mean value (default) instead of median
beat_mfcc_delta = librosa.util.sync(np.vstack([mfcc, mfcc_delta]),
                                    beat_frames)

# Compute chroma features from the harmonic signal
chromagram = librosa.feature.chroma_cqt(y=y_harmonic,
                                        sr=sr)

# Aggregate chroma features between beat events
# We'll use the median value of each feature between beat frames
beat_chroma = librosa.util.sync(chromagram,
                                beat_frames,
                                aggregate=np.median)

# Finally, stack all beat-synchronous features together
beat_features = np.vstack([beat_chroma, beat_mfcc_delta])

/usr/local/lib/python3.6/dist-packages/librosa/beat.py:306: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  hop_length=hop_length))
